### Overview 

This short lesson will demonstrate how to take a postcode or ZIP code - for example, entered by a user from a web form - and do a lookup on that postcode to get more detailed geographical data.

This data can be used to, for example, show a map of where your users are located - by transforming postcodes into `latitude` and `longitude`, finding bounding boxes for the postcode boundary, etc.

We're going to use the Nominatim OpenStreetMap API to do the lookup. Specifically, we're going to use the search functionality that the API provides - more info on this API [can be found here](https://nominatim.org/release-docs/latest/api/Search/).

To make HTTP requests to this API, we'll use the `requests` package.

In [ ]:
import requests
from pprint import pprint

Let's define the base URL for the Nominatim API. We specify `format=json` because we want the output in JSON - however, other types are available (for example, XML and GeoJSON).

In [ ]:
BASE_URL = 'https://nominatim.openstreetmap.org/search?format=json'

Let's perform a GET request to this endpoint, and pass a postcode and a zipcode to retrieve the geographical information.

In [ ]:
postcode = 'G42 9AY'

response = requests.get(f"{BASE_URL}&postalcode={postcode}")
data = response.json()
pprint(data)

[{'boundingbox': ['55.665234206593',
                  '55.985234206593',
                  '-4.413199587922',
                  '-4.093199587922'],
  'class': 'place',
  'display_name': 'Glasgow City, Scotland, G42 9AY, United Kingdom',
  'importance': 0.325,
  'lat': '55.82523420659273',
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. '
             'https://osm.org/copyright',
  'lon': '-4.253199587921969',
  'place_id': 261283313,
  'type': 'postcode'}]


Let's extract the bounding box and the latitude/longitude.

In [ ]:
latitude = data[0].get('lat')
longitude = data[0].get('lon')
print(latitude, longitude)

bbox = data[0].get('boundingbox')
print(bbox)

55.82523420659273 -4.253199587921969
['55.665234206593', '55.985234206593', '-4.413199587922', '-4.093199587922']


Now, let's look for a generic ZIP code - `2601`.



In [ ]:
zipcode = '2601'

response = requests.get(f"{BASE_URL}&postalcode={zipcode}")
response.json()

[{'boundingbox': ['45.17059985', '45.49059985', '9.614887', '9.934887'],
  'class': 'place',
  'display_name': 'Fiesco, Cremona, Lombardia, 2601, Italia',
  'importance': 0.33499999999999996,
  'lat': '45.33059985',
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'lon': '9.774886999999994',
  'place_id': 260130655,
  'type': 'postcode'},
 {'boundingbox': ['47.619425028378',
   '47.939425028378',
   '18.970696169764',
   '19.290696169764'],
  'class': 'place',
  'display_name': 'Vác, Váci járás, Pest megye, Közép-Magyarország, 2601, Magyarország',
  'importance': 0.33499999999999996,
  'lat': '47.77942502837837',
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'lon': '19.130696169763613',
  'place_id': 260097848,
  'type': 'postcode'},
 {'boundingbox': ['47.729863628182',
   '48.049863628182',
   '16.091488141533',
   '16.411488141533'],
  'class': 'place',
  'display_name': 'Gemeinde Sollenau, Bezirk Wien

We can see that there's a lot of results - many different countries have a ZIP code of 2601, including Austria, Australia, Hungary and Italy.

We can append another query parameter to narrow this down to one country - let's say Italy. Just add `&country=italia` to the URL!

In [ ]:
response = requests.get(f"{BASE_URL}&postalcode={zipcode}&country=italia")
data = response.json()

Now, we get a list with a single record, for the Italian location with ZIP code 2106.

To extract latitude and longitude, we can use the following code.

In [ ]:
latitude2 = data[0].get('lat')
longitude2 = data[0].get('lon')

print(latitude2, longitude2)

45.33059985 9.774886999999994


### Mapping the Points

Now we'll look quickly at how to plot these points on a map. We'll use Python's `folium` library to do this - it is an interface into the `Leaflet` JavaScript mapping library.

In [ ]:
import folium

# create tuples representing our location
location = float(latitude), float(longitude)
location2 = float(latitude2), float(longitude2)

# center the map at Amsterdam
amsterdam = (52.3676, 4.9041)

# create a Folium map centred at the above location
m = folium.Map(location=amsterdam, zoom_start=4, width=800, height=400)

# add markers at the locations
folium.Marker(location, popup="The postcode brought me here").add_to(m)
folium.Marker(location2, popup="The postcode brought me here").add_to(m)

# refer to the map to display it in Jupyter/Colab notebooks
m

In [ ]:
# generator expression to compute midpoint of the two locations
# this works because both locations are of form: (lat, long)
# zipping them together allows us to iterate over both lats at once, 
# and then both lons at once
midpoint_gen = ((x+y)/2 for x,y in zip(location, location2))

# convert generator to a tuple representing lat/longitude of the midpoint
midpoint = tuple(midpoint_gen)

print(location)
print(location2)
print(midpoint)

(55.82523420659273, -4.253199587921969)
(45.33059985, 9.774886999999994)
(50.57791702829637, 2.7608437060390125)


Now, let's centre map at midpoint we found above, and display all the markers for each position.

In [ ]:
import folium

# create Folium map
m = folium.Map(location=midpoint, zoom_start=4, width=800, height=400)

# add marker at the locations
folium.Marker(location, popup="The postcode brought me here").add_to(m)
folium.Marker(location2, popup="The postcode brought me here").add_to(m)
folium.Marker(midpoint, popup="Middle!").add_to(m)

m

### Find the distances between the two points

We'll use `geopy` to find the geodesic distances between the points.

In [ ]:
from geopy.distance import distance

km = distance(location, location2)
miles = distance(location, location2).miles

print("Distance between postcodes:")
print(f"{km}")
print(f"{miles} miles")

Distance between postcodes:
1527.1533016462852 km
948.929067773133 miles


### Folium bonus - Lines between points.

A small but useful feature of Folium is the ability to draw lines between points, using `folium.PolyLine()`. This is shown below, where we connect a line between the two locations.

In [ ]:
import folium

# create a Folium map centred at the above location
m = folium.Map(location=midpoint, zoom_start=4, width=800, height=400)

# add marker at the locations
folium.Marker(location, popup="The postcode brought me here").add_to(m)
folium.Marker(location2, popup="The postcode brought me here").add_to(m)

# add line between points
folium.PolyLine((location,location2)).add_to(m)

m